In [4]:
!pip install numpy 


In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import f_oneway, ttest_ind, pearsonr

Matplotlib is building the font cache; this may take a moment.


In [7]:
data = pd.read_csv("Real_Estate.csv", dtype={'Column_Name': 'str'}, low_memory=False)


In [8]:
data.head()

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.5441,Residential,Single Family,NaN,NaN,NaN,NaN


In [11]:
dtype_dict = {
    "Sale Amount": "float32",
    "List Year": "int16",
    "Town": "category",
    "Property Type": "category",
    "Assessed Value": "float32"
}

df = pd.read_csv("Real_Estate.csv", dtype=dtype_dict, low_memory=False)


In [12]:
df.isnull().sum()

Serial Number             0
List Year                 0
Date Recorded             2
Town                      0
Address                  51
Assessed Value            0
Sale Amount               0
Sales Ratio               0
Property Type        382446
Residential Type     398389
Non Use Code         784178
Assessor Remarks     926401
OPM remarks         1084598
Location             799518
dtype: int64

In [13]:
data.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
1097624    False
1097625    False
1097626    False
1097627    False
1097628    False
Length: 1097629, dtype: bool

In [14]:
df = df.drop(columns=['Assessor Remarks', 'OPM remarks', 'Location', 'Non Use Code'], errors='ignore')

print(df.columns)


Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type'],
      dtype='object')


In [15]:
df = df.dropna(subset=['Date Recorded'])


In [16]:
df['Address'] = df['Address'].fillna("Unknown Address")


In [17]:
# Assume Residential if Sale Amount is below $5M
df.loc[df['Property Type'].isna() & (df['Sale Amount'] < 5000000), 'Property Type'] = "Residential"

# Assume Commercial if Sale Amount is above $5M
df.loc[df['Property Type'].isna() & (df['Sale Amount'] >= 5000000), 'Property Type'] = "Commercial"


In [18]:
df.loc[(df['Property Type'] == "Residential") & (df['Residential Type'].isna()), 'Residential Type'] = "Single Family"


In [20]:
# Convert 'Sale Amount' and 'Assessed Value' to numeric
df['Sale Amount'] = pd.to_numeric(df['Sale Amount'], errors='coerce')
df['Assessed Value'] = pd.to_numeric(df['Assessed Value'], errors='coerce')

# Convert 'List Year' to integer
df['List Year'] = df['List Year'].astype('int16')

# Convert categorical columns to 'category' type
df['Town'] = df['Town'].astype('category')
df['Property Type'] = df['Property Type'].astype('category')
df['Residential Type'] = df['Residential Type'].astype('category')

print("Data Types Optimized!")
df.info()


Data Types Optimized!
<class 'pandas.core.frame.DataFrame'>
Index: 1097627 entries, 0 to 1097628
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype   
---  ------            --------------    -----   
 0   Serial Number     1097627 non-null  int64   
 1   List Year         1097627 non-null  int16   
 2   Date Recorded     1097627 non-null  object  
 3   Town              1097627 non-null  category
 4   Address           1097627 non-null  object  
 5   Assessed Value    1097627 non-null  float32 
 6   Sale Amount       1097627 non-null  float32 
 7   Sales Ratio       1097627 non-null  float64 
 8   Property Type     1097627 non-null  category
 9   Residential Type  1079332 non-null  category
dtypes: category(3), float32(2), float64(1), int16(1), int64(1), object(2)
memory usage: 56.5+ MB


In [22]:
print("\n Final Missing Values Count:")
print(df.isnull().sum())



 Final Missing Values Count:
Serial Number           0
List Year               0
Date Recorded           0
Town                    0
Address                 0
Assessed Value          0
Sale Amount             0
Sales Ratio             0
Property Type           0
Residential Type    18295
dtype: int64


In [23]:
print(df['Property Type'].value_counts())


Property Type
Residential       531156
Single Family     401612
Condo             105420
Two Family         26408
Three Family       12586
Commercial          8339
Vacant Land         7824
Four Family         2150
Apartments          1327
Industrial           795
Public Utility        10
Name: count, dtype: int64


In [32]:
# Fill "Single Family" where Property Type is just "Residential"
df.loc[(df['Property Type'] == "Residential") & (df['Residential Type'].isna()), 'Residential Type'] = "Single Family"



In [33]:
# Step 1: Convert 'Residential Type' to a string to avoid categorical issues
df['Residential Type'] = df['Residential Type'].astype(str)

# Step 2: Fill missing values with appropriate text-based Residential Types
df.loc[(df['Property Type'] == "Residential") & (df['Residential Type'].isna()), 'Residential Type'] = "General Residential"
df.loc[df['Property Type'] == "Single Family", 'Residential Type'] = "Single Family Home"
df.loc[df['Property Type'] == "Two Family", 'Residential Type'] = "Two-Family Home"
df.loc[df['Property Type'] == "Three Family", 'Residential Type'] = "Three-Family Home"
df.loc[df['Property Type'] == "Four Family", 'Residential Type'] = "Four-Family Home"
df.loc[df['Property Type'] == "Condo", 'Residential Type'] = "Condominium"

# Step 3: Assign a default value for other property types
df.loc[df['Property Type'].isin(["Commercial", "Industrial", "Vacant Land", "Public Utility", "Apartments"]), 'Residential Type'] = "Not Residential"

# Step 4: Convert back to categorical for optimization
df['Residential Type'] = df['Residential Type'].astype('category')

# Step 5: Verify missing values
print("Missing values in Residential Type:", df['Residential Type'].isna().sum())

# Step 6: Check unique values
print("Unique Residential Types:", df['Residential Type'].unique())


Missing values in Residential Type: 0
Unique Residential Types: ['Single Family', 'Three Family', 'Not Residential', 'Two Family', 'Condo', ..., 'Condominium', 'Two-Family Home', 'Single Family Home', 'Three-Family Home', 'Four-Family Home']
Length: 11
Categories (11, object): ['Condo', 'Condominium', 'Four Family', 'Four-Family Home', ..., 'Three Family', 'Three-Family Home', 'Two Family', 'Two-Family Home']


In [35]:
# Standardizing Residential Type values
df['Residential Type'] = df['Residential Type'].replace({
    "Condominium": "Condo",
    "Two-Family Home": "Two Family",
    "Three-Family Home": "Three Family",
    "Four-Family Home": "Four Family",
    "Single Family Home": "Single Family"
})
df['Residential Type'] = df['Residential Type'].astype('category')
print("Unique Residential Types (After Standardization):", df['Residential Type'].unique())


Unique Residential Types (After Standardization): ['Single Family', 'Three Family', 'Not Residential', 'Two Family', 'Condo', 'Four Family']
Categories (6, object): ['Condo', 'Four Family', 'Not Residential', 'Single Family', 'Three Family', 'Two Family']


In [37]:
from scipy.stats import ttest_ind

# Extract sale prices for Single-Family and Condo properties
single_family_prices = df[df['Residential Type'] == 'Single Family']['Sale Amount'].dropna()
condo_prices = df[df['Residential Type'] == 'Condo']['Sale Amount'].dropna()

# Perform t-test
t_stat, p_value = ttest_ind(single_family_prices, condo_prices, equal_var=False)

# Print results
print(f"T-statistic: {t_stat:.4f}")

print(f"P-value (full precision): {p_value:.10f}")

# Conclusion
alpha = 0.05  # 5% significance level
if p_value < alpha:
    print("✅ Reject the null hypothesis: Single-Family homes have a significantly higher average sale price than Condos.")
else:
    print("❌ Fail to reject the null hypothesis: No significant difference in average sale prices.")


T-statistic: 27.9086
P-value (full precision): 0.0000000000
✅ Reject the null hypothesis: Single-Family homes have a significantly higher average sale price than Condos.


In [38]:
from scipy.stats import f_oneway

# Extract sale prices for different Residential Types
single_family = df[df['Residential Type'] == 'Single Family']['Sale Amount'].dropna()
two_family = df[df['Residential Type'] == 'Two Family']['Sale Amount'].dropna()
three_family = df[df['Residential Type'] == 'Three Family']['Sale Amount'].dropna()
four_family = df[df['Residential Type'] == 'Four Family']['Sale Amount'].dropna()
condo = df[df['Residential Type'] == 'Condo']['Sale Amount'].dropna()

# Perform ANOVA test
f_stat, p_value = f_oneway(single_family, two_family, three_family, four_family, condo)

# Print results
print(f"F-statistic: {f_stat:.4f}, P-value: {p_value:.4f}")

# Conclusion
if p_value < alpha:
    print("✅ Reject the null hypothesis: Sale prices significantly vary across property types.")
else:
    print("❌ Fail to reject the null hypothesis: No significant difference in sale prices across property types.")


F-statistic: 760.6605, P-value: 0.0000
✅ Reject the null hypothesis: Sale prices significantly vary across property types.


In [39]:
from scipy.stats import chi2_contingency

# Create contingency table
contingency_table = pd.crosstab(df['Residential Type'], df['Assessed Value'])

# Perform Chi-Square test
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Print results
print(f"Chi-Square Statistic: {chi2_stat:.4f}, P-value: {p_value:.4f}")

# Conclusion
if p_value < alpha:
    print("✅ Reject the null hypothesis: Property Type and Assessed Value are significantly correlated.")
else:
    print("❌ Fail to reject the null hypothesis: No significant correlation between Property Type and Assessed Value.")


Chi-Square Statistic: 850933.6247, P-value: 0.0000
✅ Reject the null hypothesis: Property Type and Assessed Value are significantly correlated.


In [40]:
from scipy.stats import ttest_ind

# Define what a "Surge Pricing" sale is (e.g., top 25% of the dataset based on sale amount)
threshold = df['Sale Amount'].quantile(0.75)  # Upper 25% as 'Surge'
surge_sales = df[df['Sale Amount'] >= threshold]['Sale Amount'].dropna()
normal_sales = df[df['Sale Amount'] < threshold]['Sale Amount'].dropna()

# Perform A/B Testing (Independent T-Test)
t_stat, p_value = ttest_ind(surge_sales, normal_sales, equal_var=False)

# Print results
print(f"T-statistic: {t_stat:.4f}, P-value: {p_value:.4f}")

# Conclusion
alpha = 0.05  # 5% significance level
if p_value < alpha:
    print("✅ Reject the null hypothesis: Surge pricing significantly impacts sale amounts.")
else:
    print("❌ Fail to reject the null hypothesis: Surge pricing does NOT significantly impact sale amounts.")


T-statistic: 44.3597, P-value: 0.0000
✅ Reject the null hypothesis: Surge pricing significantly impacts sale amounts.


In [41]:
from scipy.stats import ttest_ind

# Extract two groups based on sale years (e.g., 2020 vs. 2021)
year_1_sales = df[df['List Year'] == 2020]['Sale Amount'].dropna()
year_2_sales = df[df['List Year'] == 2021]['Sale Amount'].dropna()

# Perform A/B Testing (T-Test)
t_stat, p_value = ttest_ind(year_1_sales, year_2_sales, equal_var=False)

# Print results
print(f"T-statistic: {t_stat:.4f}, P-value: {p_value:.4f}")

# Conclusion
alpha = 0.05  # 5% significance level
if p_value < alpha:
    print("✅ Reject the null hypothesis: Sale prices were significantly different between 2020 and 2021.")
else:
    print("❌ Fail to reject the null hypothesis: No significant difference in sale prices between 2020 and 2021.")


T-statistic: 0.8899, P-value: 0.3735
❌ Fail to reject the null hypothesis: No significant difference in sale prices between 2020 and 2021.


In [51]:
from scipy.stats import pearsonr

# Compute Pearson correlation
correlation, p_value = pearsonr(df['Sale Amount'].dropna(), df['Sales Ratio'].dropna())

print(f"Pearson Correlation: {correlation:.4f}, P-value: {p_value:.10f}")

# Conclusion
alpha = 0.05
if p_value < alpha:
    print("✅ Reject the null hypothesis: Higher sales amounts have significantly different sales ratios.")
else:
    print("❌ Fail to reject the null hypothesis: No significant relationship between sales amount and sales ratio.")


Pearson Correlation: -0.0004, P-value: 0.6830795686
❌ Fail to reject the null hypothesis: No significant relationship between sales amount and sales ratio.


In [1]:
pip install streamlit plotly pandas


Note: you may need to restart the kernel to use updated packages.


In [6]:
import streamlit as st
import pandas as pd
import plotly.express as px

# Load the dataset
@st.cache_data
def load_data():
    df = pd.read_csv("Real_Estate.csv", low_memory=False)  # Prevents memory optimization issues

    df["Date Recorded"] = pd.to_datetime(df["Date Recorded"], errors="coerce")
    return df

df = load_data()

# ------------------- Sidebar Filters -------------------
st.sidebar.header("Filters")
property_types = df["Property Type"].dropna().unique().tolist()  # Remove NaN


# Ensure the default values exist in the options
if property_types:  # Check if list is not empty
    property_filter = st.sidebar.multiselect("Select Property Type", property_types, default=property_types)
else:
    property_filter = st.sidebar.multiselect("Select Property Type", [])

town_filter = st.sidebar.multiselect("Select Town", df["Town"].dropna().unique(), default=df["Town"].unique())

# Apply filters
filtered_df = df[(df["List Year"].isin(year_filter)) & 
                 (df["Property Type"].isin(property_filter)) & 
                 (df["Town"].isin(town_filter))]

# ------------------- KPI Metrics -------------------
total_sales = filtered_df["Sale Amount"].sum()
average_price = filtered_df["Sale Amount"].mean()
property_count = filtered_df["Address"].nunique()


col1, col2, col3 = st.columns(3)
col1.metric("Total Sales ($)", f"{total_sales:,.0f}")
col2.metric("Average Sale Price ($)", f"{average_price:,.0f}")
col3.metric("Total Properties Sold", f"{property_count:,}")

# ------------------- Plot 1: Sales Trend -------------------
st.subheader("📈 Sales Trend Over Time")
sales_trend = filtered_df.groupby("List Year")["Sale Amount"].sum().reset_index()
fig1 = px.line(sales_trend, x="List Year", y="Sale Amount", title="Total Sales Over Years", markers=True)
st.plotly_chart(fig1, use_container_width=True)

# ------------------- Plot 2: Property Type Distribution -------------------
st.subheader("🏡 Property Type Distribution")
property_count_df = filtered_df["Property Type"].value_counts().reset_index()
property_count_df.columns = ["Property Type", "Count"]
fig2 = px.pie(property_count_df, values="Count", names="Property Type", title="Distribution of Property Types")
st.plotly_chart(fig2, use_container_width=True)

# ------------------- Plot 3: Sales Amount by Town -------------------
st.subheader("🏙️ Sales by Town")
sales_by_town = filtered_df.groupby("Town")["Sale Amount"].sum().reset_index()
fig3 = px.bar(sales_by_town, x="Sale Amount", y="Town", title="Total Sales per Town", orientation="h")
st.plotly_chart(fig3, use_container_width=True)

# ------------------- Plot 4: Assessed Value vs. Sale Price -------------------
st.subheader("💰 Assessed Value vs. Sale Price")
fig4 = px.scatter(filtered_df, x="Assessed Value", y="Sale Amount", color="Property Type",
                  title="Assessed Value vs. Sale Price", trendline="ols")
st.plotly_chart(fig4, use_container_width=True)

# ------------------- Summary -------------------
st.markdown("""
### 🔍 Key Insights:
1. **Sales Trends:** Check how real estate sales are fluctuating over time.
2. **Property Distribution:** See which types of properties are most common.
3. **Sales by Town:** Understand which towns contribute the most to sales.
4. **Value vs. Price:** Compare assessed value with the actual sale price.
""")


2025-03-18 09:06:45.612 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()

In [4]:
df.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type', 'Non Use Code', 'Assessor Remarks', 'OPM remarks',
       'Location'],
      dtype='object')